# Практическое занятие № 3 по проверке гипотез

## Шпаргалка по выбору теста

На что обратить внимание:
1. Тип данных: continuous / categorical.
2. Пары или независимые наблюдения.
3. Число групп.
4. Проверка нормальности (Shapiro) и однородности дисперсий (Levene).

**Ещё одна, более расширенная шпаргалка по выбору критерия тестирования:**

| Критерий / тест                      |                                                                                Когда применять | Функция (scipy или альтернатива)                                                        |
| ------------------------------------ | ------------------------------------------------------------------------------------------------------: | ------------------------------------------------------------------------------------- |
| Shapiro–Wilk (проверка нормальности) |                                                   Проверка нормальности одной выборки (малые/средние n) | `scipy.stats.shapiro`                                                                 |
| Одновыборочный t-тест (one-sample t) |                       Проверить, равно ли среднее заданному значению, σ неизвестна, approx нормальность | `scipy.stats.ttest_1samp`                                                             |
| Двухвыборочный t (Student, pooled)   |                            Сравнить средние двух независимых групп при нормальности и равных дисперсиях | `scipy.stats.ttest_ind(..., equal_var=True)`                                          |
| Welch t                  |                        Сравнить средние двух независимых групп при нормальности, но неравных дисперсиях | `scipy.stats.ttest_ind(..., equal_var=False)`                                         |
| Paired t (парный t-тест)             |                                                    Парные наблюдения (до/после), нормальность разностей | `scipy.stats.ttest_rel`                                                               |
| Mann–Whitney U                       |                             Две независимые выборки, ненормальные или порядковые данные (тест на сдвиг) | `scipy.stats.mannwhitneyu`                                                            |
| Wilcoxon signed-rank                 |                                                      Парные ненормальные данные (альтернатива paired t) | `scipy.stats.wilcoxon`                                                                |
| One-way ANOVA                        |                                    Сравнение средних >2 групп при нормальности и однородности дисперсий | `scipy.stats.f_oneway`                                                                |
| Kruskal–Wallis                       |                                                       Непараметрическая альтернатива ANOVA для >2 групп | `scipy.stats.kruskal`                                                                 |
| $\chi^2$ — goodness-of-fit                 |                                Сравнить наблюдаемые частоты с ожидаемыми по распределению (categorical) | `scipy.stats.chisquare`                                                               |
| $\chi^2$ — contingency (independence)      |                                              Проверка зависимости/независимости в таблице сопряжённости | `scipy.stats.chi2_contingency`                                                        |
| Fisher exact (точный)                |                                                    $2\times 2$ таблица при малых ожидаемых счётах — точный тест | `scipy.stats.fisher_exact`                                                            |
| Binomial test (для доли)             |                                                               Проверить H0 для одной пропорции (точный) | `scipy.stats.binom_test` **(старое)** / `scipy.stats.binomtest` **(новое)**           |
| Z-test для разности пропорций        |                                     Сравнить две пропорции при достаточном n (A/B) — стандартный подход | **в SciPy нет готовой**; используйте `statsmodels.stats.proportion.proportions_ztest` |
| Levene (равенство дисперсий)         |                                              Тест на однородность дисперсий (робастен к ненормальности) | `scipy.stats.levene`                                                                  |
| Bartlett (равенство дисперсий)       |                                          Тест на однородность дисперсий (чувствителен к ненормальности) | `scipy.stats.bartlett`                                                                |
| Kolmogorov–Smirnov (KS)              | 1) проверка соответствия выборки указанному распределению (kstest) 2) сравнение двух выборок (ks_2samp) | `scipy.stats.kstest`, `scipy.stats.ks_2samp`                                          |


**Непрерывные / порядковые (continuous / ordinal):**
Shapiro–Wilk, t-test (one-sample, two-sample Student / Welch), paired t, ANOVA (f_oneway), Mann–Whitney U, Wilcoxon signed-rank, Kruskal–Wallis, Levene, Bartlett, Kolmogorov–Smirnov.

**Дискретные / категориальные / счётчики (discrete / counts / proportions):**
$\chi^2$ goodness-of-fit, $\chi^2$ contingency, Fisher exact ($2\times 2$), binomial/binomtest, proportions z-test (две пропорции / AB-тесты), тесты для Пуассона.

**Гибрид / с оговорками (подходят для порядковых или требуют осторожности):**
Mann–Whitney и Wilcoxon — работают с порядковыми данными и с дискретными шкалами (но не для частот и таблиц сопряженности); KS — ориентирован на непрерывные распределения (для дискретных данных даёт некорректные p-values без поправок).

*Итог:* если данные **частоты/доли** - смотрите $\chi^2$ / Fisher / binom / proportions; если **непрерывные или порядковые оценки** -> смотрите t / ANOVA / (или непараметрические) Mann-Whitney, Wilcoxon, Kruskal-Wallis; некоторые тесты (KS, Mann-Whitney) можно применять и к порядковым/дискретным данным с оговорками.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportion_confint
import matplotlib.pyplot as plt
from math import isnan
from scipy.stats import fisher_exact

np.random.seed(42)

/Users/maria/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/maria/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Задача 1. Одновыборочный t-тест

Менеджер продукта хочет проверить, действительно ли средняя оценка нового фич-релиза равна заявленной 7. Собрали выборку из 40 пользователей, каждый дал оценку от 0 до 10.

Нужно формально проверить гипотезу $H_0:\; \mu = 7$ на уровне $\alpha=0.05$ и дать $95\%$ доверительный интервал для среднего.

In [ ]:
# generate sample
sample = np.random.normal(loc=6.8, scale=1.5, size=40)

# Shapiro-Wilk для проверки нормальности (на n=40 применим)
sh_stat, sh_p = stats.shapiro(sample)

# one-sample t-test H0: mu = 7
t_stat, two_sided_p = stats.ttest_1samp(sample, 7.0)
one_sided_p = two_sided_p / 2 if t_stat < 0 else 1 - two_sided_p / 2

# 95% CI
mean = sample.mean()
se = stats.sem(sample)
ci = stats.t.interval(0.95, df=len(sample)-1, loc=mean, scale=se)

# минимальный вывод
result_task1 = dict(shapiro_p=sh_p, t_stat=t_stat, one_sided_p=one_sided_p, ci=ci)
result_task1

{'shapiro_p': 0.36431223154067993,
 't_stat': -0.9168552520862595,
 'one_sided_p': 0.1824268167448097,
 'ci': (6.346972862741551, 7.245664008950235)}

Почему этот критерий? $\sigma$ неизвестна, $n=40$ — стандартный одновыборочный t-тест подходит при приблизительной нормальности выборки. Shapiro проверяет нормальность, а CI показывает практическую значимость.

### Задача 2. Тест для доли

Маркетолог сообщает, что $\geq70\%$ покупателей купят товар снова. Из случайной выборки 200 покупателей 130 подтвердили повторную покупку.

Проверить $H_0:\; p = 0.7$ против альтернативы $p < 0.7$ и дать $95\%$ CI для доли.

In [ ]:
# help(proportions_ztest)

In [ ]:
n = 200
k = 130

# exact binomial p-value (H1: p < 0.7)
exact_p = stats.binom_test(k, n, p=0.7, alternative='less')

# z-test (приближение) и Wilson CI
z_stat, z_p = proportions_ztest([k], [n], value=0.7, alternative='smaller')
ci_lower, ci_upper = proportion_confint(k, n, method='wilson')

result_task2 = dict(exact_p=exact_p, z_p=z_p, z_stat=z_stat, ci=(ci_lower, ci_upper))
result_task2

/var/folders/8w/sx6j7f5d7td7tzr3k347hzkc0000gn/T/ipykernel_6341/101416171.py:5: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  exact_p = stats.binom_test(k, n, p=0.7, alternative='less')


{'exact_p': 0.07278645724053477,
 'z_p': array([0.06910383]),
 'z_stat': array([-1.48249863]),
 'ci': (0.5816346433604008, 0.7127117587264192)}

Почему этот критерий? Для проверки доли можно использовать точный биномиальный тест (точен при любых n). Для больших n z-approx даёт быстрый результат; Wilson CI даёт более корректный доверительный интервал для пропорций.

### Задача 3. Критерий согласия $\chi^2$ (goodness-of-fit)

Мерчендайзеры считают, что просмотры четырёх витрин распределены равномерно. За день собрано 100 наблюдений: [20, 30, 25, 25].

Нужно проверить соответствие равномерной модели (H0: все категории равновероятны).

In [ ]:
obs = np.array([20, 30, 25, 25])
expected = np.full_like(obs, obs.sum() / obs.size, dtype=float)

chi2_stat, p_value = stats.chisquare(obs, f_exp=expected)

result_task3 = dict(chi2=chi2_stat, p=p_value, expected=expected)
result_task3

{'chi2': 2.0, 'p': 0.5724067044708798, 'expected': array([25., 25., 25., 25.])}

Почему этот критерий? $\chi^2$ goodness-of-fit сравнивает наблюдаемые и ожидаемые частоты. Важно проверять, что ожидаемые частоты достаточно большие (обычно $\geq5$) — здесь это выполняется.

### Задача 4. Две независимые нормальные выборки (Student t vs Welch)

Маркетинговые кампании A и B дали разные продажи за неделю. Собрали случайные выборки продаж: группа A (n=45), группа B (n=50).

Нужно проверить, отличаются ли средние продажи между A и B. Перед тестированием проверить нормальность и однородность дисперсий.

In [ ]:
groupA = np.random.normal(50, 5, size=45)
groupB = np.random.normal(53, 7, size=50)

# Проверки предпосылок
sh_A = stats.shapiro(groupA)[1]
sh_B = stats.shapiro(groupB)[1]
levene_p = stats.levene(groupA, groupB)[1]

# Выбор теста: если var равны -> объединённый t, иначе Welch
if levene_p > 0.05:
    t_res = stats.ttest_ind(groupA, groupB, equal_var=True)
    test_used = "student_pooled"
else:
    t_res = stats.ttest_ind(groupA, groupB, equal_var=False)
    test_used = "welch"

result_task4 = dict(shapiro_p=(sh_A, sh_B), levene_p=levene_p, test=test_used, stat=t_res.statistic, p=t_res.pvalue)
result_task4

Почему этот критерий? Если дисперсии однородны (Levene p > 0.05) — используем pooled Student t (более мощный). Если дисперсии различаются — Welch t корректирует степень свободы и даёт более надёжный результат. Нормальность проверяется Shapiro; при выраженном отклонении от нормальности стоит заменить тест на непараметрический (Mann–Whitney).

### Задача 5. Парные выборки (paired t-test / Wilcoxon)

Клиническое испытание: у 30 пациентов замеряли артериальное давление до и через 1 месяц после лекарства. Требуется проверить, снизилось ли давление (парные наблюдения), и если распределение разностей ненормально — применить непараметрическую альтернативу.

In [ ]:
before = np.random.normal(140, 10, size=30)
after = before - np.random.normal(3.0, 5.0, size=30)

# нормальность разностей
diff = after - before
sh_diff_p = stats.shapiro(diff)[1]

if sh_diff_p > 0.05:
    res = stats.ttest_rel(before, after)
    test_used = "paired_t"
else:
    res = stats.wilcoxon(before, after)
    test_used = "wilcoxon_signed_rank"

result_task5 = dict(test=test_used, stat=getattr(res, 'statistic', None), p=getattr(res, 'pvalue', getattr(res, 'p', None)))
result_task5

{'test': 'paired_t', 'stat': 4.079159168339154, 'p': 0.00032255223198430203}

### Задача 6. Mann–Whitney U для двух независимых ненормальных выборок

Команда UX запустила два варианта интерфейса. Для случайной выборки пользователей собрали оценки удовлетворённости (шкала 1–10). Распределение оценок похоже на экспоненциальное (сильный сдвиг).

Нужно проверить, отличаются ли оценки между вариантами, без допущения нормальности.

In [ ]:
g1 = np.random.exponential(scale=2.0, size=40)
g2 = np.random.exponential(scale=1.6, size=45)

u_stat, p_value = stats.mannwhitneyu(g1, g2, alternative='two-sided')
result_task6 = dict(u=u_stat, p=p_value)
result_task6

{'u': 1135.0, 'p': 0.038955430345207924}

Почему этот критерий? Данные явно ненормальны и независимы -> Mann–Whitney U тест (непарметрический) проверяет, есть ли сдвиг в распределениях (часто интерпретируется как разница медиан).

### Задача 7. ANOVA и Kruskal-Wallis для >2 групп
Университет тестирует три обучающие программы; для каждой взята группа студентов и измерен итоговый балл.

Нужно проверить, есть ли статистически значимые различия между средними трёх групп, а при нарушении предпосылок — использовать непараметрическую альтернативу.

In [ ]:
g1 = np.random.normal(70, 6, size=30)
g2 = np.random.normal(73, 6, size=28)
g3 = np.random.normal(69, 7, size=32)

anova_f, anova_p = stats.f_oneway(g1, g2, g3)
kw_h, kw_p = stats.kruskal(g1, g2, g3)
result_task7 = dict(anova_p=anova_p, kruskal_p=kw_p)
result_task7

{'anova_p': 0.03851270211966855, 'kruskal_p': 0.0476654777175092}

Почему эти критерии? ANOVA — стандарт для сравнения средних >2 групп при нормальности/однородности. Kruskal–Wallis — непараметрическая альтернатива, когда предпосылки нарушены.

### Задача 8. $\chi^2$ для таблицы сопряжённости

Исследователь сравнивает выбор курсов (A/B/C) и категорию успеваемости (low/med/high) на выборке студентов; данные в виде $3\times3$ таблицы.

Нужно проверить, связаны ли курс и успеваемость.

In [ ]:
obs = np.array([[20, 30, 10],
                [25, 20, 20],
                [10, 15, 25]])

chi2_stat, p_value, dof, expected = stats.chi2_contingency(obs)
result_task8 = dict(chi2=chi2_stat, p=p_value, dof=dof, expected=expected)
result_task8

{'chi2': 16.89438766361844,
 'p': 0.0020264372371853515,
 'dof': 4,
 'expected': array([[18.85714286, 22.28571429, 18.85714286],
        [20.42857143, 24.14285714, 20.42857143],
        [15.71428571, 18.57142857, 15.71428571]])}

Почему этот критерий? $\chi^2$ тест независимости сравнивает наблюдаемые и ожидаемые частоты в таблице; при небольших ожидаемых значениях (в ячейках <5) нужно либо объединять категории, либо для $2\times 2$ применять Fisher exact.

### Задача 9. Доверительный интервал для разности средних

Сеть магазинов сравнивает средние продажи в двух регионах; дисперсии могут отличаться.

Нужно оценить разность средних и построить 95% CI (Welch-подход), чтобы понять практическую значимость.

In [ ]:
a = np.random.normal(100, 10, size=60)
b = np.random.normal(103, 10, size=55)

mean_diff = a.mean() - b.mean()
se_diff = (a.var(ddof=1)/len(a) + b.var(ddof=1)/len(b))**0.5

sa = a.var(ddof=1)/len(a)
sb = b.var(ddof=1)/len(b)
df = (sa + sb)**2 / ((sa**2/(len(a)-1)) + (sb**2/(len(b)-1)))
t_crit = stats.t.ppf(0.975, df)
ci = (mean_diff - t_crit * se_diff, mean_diff + t_crit * se_diff)

result_task9 = dict(mean_diff=mean_diff, ci=ci)
result_task9

{'mean_diff': -0.03321283566573641,
 'ci': (-3.8140583480499974, 3.7476326767185246)}

Почему этот подход? Welch даёт корректный SE и df при неравных дисперсиях; CI показывает границы эффекта — важно для практической интерпретации (не только p-value).

### Задача 10. Тесты на равенство дисперсий: Levene vs Bartlett

Инженер сравнивает вариативность размеров деталей с двух линий.

Нужно проверить гипотезу равенства дисперсий, чтобы выбрать последующий t-test с корректной формой.

In [ ]:
s1 = np.random.normal(10, 2.0, size=40)
s2 = np.random.normal(10, 4.0, size=40)

levene_stat, levene_p = stats.levene(s1, s2)
bartlett_stat, bartlett_p = stats.bartlett(s1, s2)

result_task10 = dict(levene_p=levene_p, bartlett_p=bartlett_p)
result_task10

{'levene_p': 7.128669668068426e-05, 'bartlett_p': 1.1467229071467553e-07}

Почему эти критерии? Bartlett чувствителен к ненормальности; Levene более робастен и предпочтителен в практических задачах. Сравнение обоих помогает интерпретировать результаты.

### Задача 11. Fisher exact для $2\times 2$ при малых частотах

Пилотное исследование (маленькая выборка) сравнивает наличие побочного эффекта в двух группах лечения; таблица $2\times 2$ содержит низкие счётности.

Нужно проверить связь лечения и эффекта.

In [ ]:
obs = np.array([[1, 8],
                [5, 2]])

oddsratio, p_value = stats.fisher_exact(obs, alternative='two-sided')
result_task11 = dict(oddsratio=oddsratio, p=p_value)
result_task11

{'oddsratio': 0.05, 'p': 0.03496503496503496}

Почему этот критерий? При малых частотах $\chi^2$ ненадёжен; Fisher exact даёт точный p-value для $2\times 2$ таблиц.

### Задача 12. Wilcoxon signed-rank для парных небольших ненормальных данных
Пилот с 12 участниками: оценка до и после изменения интерфейса. Данные явно ненормальны (сильный сдвиг).

Нужно проверить, изменилась ли оценка у участников.

In [ ]:
np.random.seed(9)
before = np.random.exponential(scale=2.0, size=12)
after = before - np.random.exponential(scale=0.5, size=12)

w_stat, p_value = stats.wilcoxon(before, after)
result_task12 = dict(w_stat=w_stat, p=p_value)
result_task12

{'w_stat': 0.0, 'p': 0.00048828125}

Почему этот критерий? При парных наблюдениях и малом n/ненормальности Wilcoxon signed-rank — подходящая непараметрическая альтернатива paired t-test.

### Задача 13. A/B тест: сравнение двух пропорций
В веб-эксперименте: конверсия A = 120/2000, B = 150/2100. Менеджер хочет знать, есть ли статистически значимая разница в конверсиях и получить 95% CI для каждой доли.

In [ ]:
count = np.array([120, 150])
nobs = np.array([2000, 2100])

z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')
ci_A = proportion_confint(count[0], nobs[0], method='wilson')
ci_B = proportion_confint(count[1], nobs[1], method='wilson')

result_task13 = dict(z=z_stat, p=p_value, ci_A=ci_A, ci_B=ci_B)
result_task13

{'z': -1.4747800411431655,
 'p': 0.14027168882592367,
 'ci_A': (0.0504111971143165, 0.07127580449318606),
 'ci_B': (0.06117844577266071, 0.08324377651352433)}

Почему этот критерий? Для сравнений двух независимых пропорций при достаточном n z-test — стандартный инструмент; Wilson CI даёт надёжные интервалы для пропорций.